In [1]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [3]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [4]:
from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import _fetch_batch, fetch_data, fetch_spot_data

In [5]:
from expiries import dict_expiries

In [6]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("o").first().alias("o"),
                pl.col("h").max().alias("h"),
                pl.col("l").min().alias("l"),
                pl.col("c").last().alias("c"),
                # pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [7]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2019, 2026),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe

    # Iterate over each year
    for year in range(2019, 2026):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()

        # Calculate total number of trades
        total_trades = len(year_trades)

        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100

        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()

        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()

        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()

        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = f"{variation}"

        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]

    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0][
        "ROI%"
    ].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum()
        - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation

    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio: stats_df8}

In [8]:
bnf_1min = pd.read_csv("../data/nifty_min.csv")

In [9]:
bnf_1min.columns = ['index', 'datetime', 'o', 'h', 'l', 'c', 'v']
bnf_1min.tail()

,index,datetime,o,h,l,c,v
750760,nifty,2025-03-28 15:25:00,23500.2500,23502.5000,23485.7500,23490.4000,0
750761,nifty,2025-03-28 15:26:00,23490.7500,23494.3500,23486.7500,23494.0500,0
750762,nifty,2025-03-28 15:27:00,23494.2000,23497.4500,23489.7500,23496.8000,0
750763,nifty,2025-03-28 15:28:00,23497.1000,23500.4500,23491.0500,23492.0000,0
750764,nifty,2025-03-28 15:29:00,23492.2500,23503.1000,23450.2000,23495.1500,0


In [10]:
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"]).dt.tz_localize(None)
# bnf_1min = bnf_1min[((bnf_1min['datetime'].dt.year == 2020) & (bnf_1min['datetime'].dt.month == 4))]
bnf_1min = bnf_1min[
    (bnf_1min["datetime"].dt.year >= 2019) & (bnf_1min["datetime"].dt.year <= 2025)
]

In [11]:
dict_expiries

{'nifty': [datetime.datetime(2017, 1, 25, 0, 0),
  datetime.datetime(2017, 2, 23, 0, 0),
  datetime.datetime(2017, 3, 30, 0, 0),
  datetime.datetime(2017, 4, 27, 0, 0),
  datetime.datetime(2017, 5, 25, 0, 0),
  datetime.datetime(2017, 6, 29, 0, 0),
  datetime.datetime(2017, 7, 27, 0, 0),
  datetime.datetime(2017, 8, 31, 0, 0),
  datetime.datetime(2017, 9, 28, 0, 0),
  datetime.datetime(2017, 10, 26, 0, 0),
  datetime.datetime(2017, 11, 30, 0, 0),
  datetime.datetime(2017, 12, 28, 0, 0),
  datetime.datetime(2018, 1, 25, 0, 0),
  datetime.datetime(2018, 2, 22, 0, 0),
  datetime.datetime(2018, 3, 28, 0, 0),
  datetime.datetime(2018, 4, 26, 0, 0),
  datetime.datetime(2018, 5, 31, 0, 0),
  datetime.datetime(2018, 6, 28, 0, 0),
  datetime.datetime(2018, 7, 26, 0, 0),
  datetime.datetime(2018, 8, 30, 0, 0),
  datetime.datetime(2018, 9, 27, 0, 0),
  datetime.datetime(2018, 10, 25, 0, 0),
  datetime.datetime(2018, 11, 29, 0, 0),
  datetime.datetime(2018, 12, 27, 0, 0),
  datetime.datetime(2019,

In [12]:
from datetime import date
from bisect import bisect_left

def get_next_expiry(input_date, index_symbol):
    expiries = dict_expiries.get(index_symbol)
    if not expiries:
        return None
        
    expiry_dates = sorted({dt.date() for dt in expiries})
    pos = bisect_left(expiry_dates, input_date)
    return expiry_dates[pos] if pos < len(expiry_dates) else None


In [13]:
index_ = 'nifty'

if index_ == 'nifty':
    LOT_SIZE_ = 75
    STRIKE_SPREAD_ = 50
    INDEX_LEVERAGE_ = 8
    PORTFOLIO_ = 1_00_00_000

In [14]:
import pandas as pd

async def add_atr(df, period=14):
    """
    Adds an 'ATR' column to the DataFrame using Wilder's smoothing (like TradingView).
    
    Parameters:
    df (pd.DataFrame): Must contain 'h', 'l', 'c' columns for high, low, close
    period (int): ATR period (default 14)
    
    Returns:
    pd.DataFrame: With 'ATR' column added
    """
    high = df['h']
    low = df['l']
    close = df['c']

    # True Range
    tr1 = high - low
    tr2 = (high - close.shift()).abs()
    tr3 = (low - close.shift()).abs()
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)

    # ATR with Wilder's smoothing (like an EMA with alpha=1/period)
    atr = tr.ewm(alpha=1/period, adjust=False).mean()

    df['ATR'] = atr
    return df


In [29]:
async def backtest_intraday_levels(df, multiplier, tf, offset):
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df[df['datetime'].dt.year >= 2025]
    df['date'] = df['datetime'].dt.date
    # print(df.head().to_string())
    
    tradebook = []

    eod_time = dt.time(15, 20)

    for date, group in df.groupby('date'):
        current_date = date
        group = group.reset_index(drop=True)
        # print(date)
        # Get 9:15 candle
        morning_candle = group[group['datetime'].dt.time == pd.to_datetime("09:15").time()]
        if morning_candle.empty:
            continue

        morning_atr = morning_candle.iloc[0]['ATR']
        running_high = group.iloc[0]['h']
        running_low = group.iloc[0]['l']

        is_high_breached = False
        is_low_breached = False

        in_trade_long = False
        in_trade_short = False

        data_fetched_pe = False
        data_fetched_ce = False

        for i in range(0, len(group)):
            row = group.iloc[i]
            running_high = max(running_high, row['h'])
            running_low = min(running_low, row['l'])
            current_datetime = row['datetime']
            
            high_level = running_low + (multiplier * morning_atr)
            low_level = running_high - (multiplier * morning_atr)
            
            # print(row)
            print(group.iloc[i]['datetime'], high_level, low_level)
            print(f'RH : {running_high} , RL : {running_low}')

            high_strike = int(round(high_level / STRIKE_SPREAD_) * STRIKE_SPREAD_)
            low_strike = int(round(low_level / STRIKE_SPREAD_) * STRIKE_SPREAD_)

            expiry = get_next_expiry(date, index_)
            dte = (expiry - current_date).days

            if not is_high_breached and row['c'] > high_level and i>0 and not in_trade_long and current_datetime.time() <= eod_time:
                # print('High Breached')
                
                if not data_fetched_pe:
                    pe_df = await fetch_data(
                        index=index_,
                        expiry=expiry,
                        strike=high_strike,
                        asset_class='P',
                        start_date=date,
                        start_time=dt.time(9, 15),
                        end_date=date,
                        end_time=dt.time(15, 30),
                    )
                    data_fetched_pe = True

                if data_fetched_pe:
                    if pe_df is not None and not isinstance(pe_df, str):
                        if (len(pe_df) * int(tf[:-1])) > (200 / int(tf[:-1])):
                            pe_df = resample(pe_df, tf, offset)
                            pe_df_pandas = pe_df.to_pandas()
                            subset_df = pe_df_pandas[pe_df_pandas['datetime'] >= current_datetime]
                            pe_entry_price = subset_df.iloc[0]['c']
                            print('Long Trade Entered')
                        else:
                            pe_entry_price = float('nan')
                    else:
                        pe_entry_price = float('nan')

                trade = {
                    'date': current_date,
                    'high level': high_level,
                    'low level': low_level,
                    'atr_multiplier': multiplier,
                    'Morning ATR': morning_atr,
                    'side': 'LONG',
                    'strike': high_strike,
                    'type': 'PE',
                    'expiry': expiry,
                    'dte': dte,
                    'entry_time': current_datetime,
                    'entry price': pe_entry_price,
                }
                    
                tradebook.append(trade)
                is_high_breached = True
                in_trade_long = True
                # continue

            if in_trade_long:
                if row['c'] <= low_level:
                    # TSL Hit
                    for trade in tradebook:
                        if (
                            trade['type'] == 'PE' and
                            trade['date'] == current_date and
                            'exit_time' not in trade
                        ):
                            df_row = pe_df_pandas[pe_df_pandas['datetime'] <= current_datetime]
                            # print(df_row.to_string())
                            pe_exit_price = df_row['c'].iloc[-1] if len(df_row) != 0 else float('nan')
                            trade['exit price'] = pe_exit_price
                            trade['exit_time'] = df_row['datetime'].iloc[-1] if len(df_row) != 0 else float('nan')
                            trade['remarks'] = 'TSL Hit'
                            trade['points'] = trade['entry price'] - trade['exit price']
                            print('Long TSL Hit')
                    
                    in_trade_long = False
                    is_high_breached = False

                    running_low = row['l']

                elif current_datetime.time() >= eod_time:
                    # EOD Exit
                    for trade in tradebook:
                        if (
                            trade['type'] == 'PE' and
                            trade['date'] == current_date and
                            'exit_time' not in trade
                        ):
                            df_row = pe_df_pandas[pe_df_pandas['datetime'] <= current_datetime]
                            # print(df_row.to_string())
                            pe_exit_price = df_row['c'].iloc[-1] if len(df_row) != 0 else float('nan')
                            trade['exit price'] = pe_exit_price
                            trade['exit_time'] = df_row['datetime'].iloc[-1] if len(df_row) != 0 else float('nan')
                            trade['remarks'] = 'EOD Exit'
                            trade['points'] = trade['entry price'] - trade['exit price']

                    in_trade_long = False
                    is_high_breached = False
            
            if not is_low_breached and row['c'] < low_level and i>0 and not in_trade_short and current_datetime.time() <= eod_time:
                
                if not data_fetched_ce:
                    ce_df = await fetch_data(
                        index=index_,
                        expiry=expiry,
                        strike=low_strike,
                        asset_class='C',
                        start_date=date,
                        start_time=dt.time(9, 15),
                        end_date=date,
                        end_time=dt.time(15, 30),
                    )
                    data_fetched_ce = True

                if data_fetched_ce:
                    if ce_df is not None and not isinstance(ce_df, str):
                        if (len(ce_df) * int(tf[:-1])) > (200 / int(tf[:-1])):
                            ce_df = resample(ce_df, tf, offset)
                            ce_df_pandas = ce_df.to_pandas()
                            subset_df = ce_df_pandas[ce_df_pandas['datetime'] >= current_datetime]
                            ce_entry_price = subset_df.iloc[0]['c']
                            print('Short Trade Entered')
                        else:
                            ce_entry_price = float('nan')
                    else:
                        ce_entry_price = float('nan')

                trade = {
                    'date': current_date,
                    'high level': high_level,
                    'low level': low_level,
                    'atr_multiplier': multiplier,
                    'Morning ATR': morning_atr,
                    'side': 'SHORT',
                    'strike': low_strike,
                    'type': 'CE',
                    'expiry': expiry,
                    'dte': dte,
                    'entry_time': current_datetime,
                    'entry price': ce_entry_price,
                }
                    
                tradebook.append(trade)
                is_low_breached = True
                in_trade_short = True
                # continue

            if in_trade_short:
                print(f'Row Close : {row["c"]} , High Level : {high_level}')
                if row['c'] >= high_level:
                    # TSL Hit
                    for trade in tradebook:
                        if (
                            trade['type'] == 'CE' and
                            trade['date'] == current_date and
                            'exit_time' not in trade
                        ):
                            df_row = ce_df_pandas[ce_df_pandas['datetime'] <= current_datetime]
                            # print(df_row.to_string())
                            ce_exit_price = df_row['c'].iloc[-1] if len(df_row) != 0 else float('nan')
                            trade['exit price'] = ce_exit_price
                            trade['exit_time'] = df_row['datetime'].iloc[-1] if len(df_row) != 0 else float('nan')
                            trade['remarks'] = 'TSL Hit'
                            trade['points'] = trade['entry price'] - trade['exit price']
                    
                    in_trade_short = False
                    is_low_breached = False

                    running_high = row['h']

                elif current_datetime.time() >= eod_time:
                    # EOD Exit
                    for trade in tradebook:
                        if (
                            trade['type'] == 'CE' and
                            trade['date'] == current_date and
                            'exit_time' not in trade
                        ):
                            df_row = ce_df_pandas[ce_df_pandas['datetime'] <= current_datetime]
                            # print(df_row.to_string())
                            ce_exit_price = df_row['c'].iloc[-1] if len(df_row) != 0 else float('nan')
                            trade['exit price'] = ce_exit_price
                            trade['exit_time'] = df_row['datetime'].iloc[-1] if len(df_row) != 0 else float('nan')
                            trade['remarks'] = 'EOD Exit'
                            trade['points'] = trade['entry price'] - trade['exit price']
                            print('Short TSL Hit')

                    in_trade_short = False
                    is_low_breached = False

    tb = pd.DataFrame(tradebook)
    if len(tb) > 0:
        tb['slippage'] = 0.01 * (tb['entry price'] + tb['exit price'])
        tb['final_points'] = tb['points'] - tb['slippage']
        tb['portfolio'] = PORTFOLIO_
        tb['index leverage'] = INDEX_LEVERAGE_
        tb['qty'] = tb['portfolio'] * tb['index leverage'] / tb['strike']
        tb['pnl'] = tb['final_points'] * tb['qty']
        tb['ROI%'] = tb['pnl'] * 100 / tb['portfolio']
        tb['Trade Year'] = tb['entry_time'].dt.year
    return tb


In [30]:
TF_ = '3m'
OFFSET_ = '0m'
ATR_WINDOW_ = 21
MULTIPLIER_ = 5
bnf_df = resample(pl.DataFrame(bnf_1min), TF_, OFFSET_)
bnf_df_pandas = bnf_df.to_pandas()
bnf_df_pandas = await add_atr(bnf_df_pandas, ATR_WINDOW_)
tb = await backtest_intraday_levels(bnf_df_pandas, MULTIPLIER_, TF_, OFFSET_)
# tb

/tmp/ipykernel_982/564542651.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['datetime'].dt.date


2025-01-01 09:15:00 23704.823269403987 23583.926730596013
RH : 23681.7 , RL : 23607.05
2025-01-01 09:18:00 23704.823269403987 23585.82673059601
RH : 23683.6 , RL : 23607.05
2025-01-01 09:21:00 23704.823269403987 23585.82673059601
RH : 23683.6 , RL : 23607.05
2025-01-01 09:24:00 23704.823269403987 23585.82673059601
RH : 23683.6 , RL : 23607.05
2025-01-01 09:27:00 23704.823269403987 23585.82673059601
RH : 23683.6 , RL : 23607.05
2025-01-01 09:30:00 23704.823269403987 23585.82673059601
RH : 23683.6 , RL : 23607.05
2025-01-01 09:33:00 23704.823269403987 23585.82673059601
RH : 23683.6 , RL : 23607.05
2025-01-01 09:36:00 23704.823269403987 23585.82673059601
RH : 23683.6 , RL : 23607.05
2025-01-01 09:39:00 23704.823269403987 23585.82673059601
RH : 23683.6 , RL : 23607.05
2025-01-01 09:42:00 23696.17326940399 23585.82673059601
RH : 23683.6 , RL : 23598.4
2025-01-01 09:45:00 23671.073269403987 23585.82673059601
RH : 23683.6 , RL : 23573.3
Short Trade Entered
Row Close : 23580.45 , High Level : 

In [31]:
tb.tail(22)

,date,high level,low level,atr_multiplier,Morning ATR,side,strike,type,expiry,dte,entry_time,entry price,exit price,exit_time,remarks,points,slippage,final_points,portfolio,index leverage,qty,pnl,ROI%,Trade Year
129,2025-03-19,22872.6553,22823.5947,5,12.9411,SHORT,22800,CE,2025-03-20,1,2025-03-19 09:24:00,116.5500,140.1000,2025-03-19 09:51:00,TSL Hit,-23.5500,2.5665,-26.1165,10000000,8,3508.7719,-91636.8421,-0.9164,2025
130,2025-03-19,22872.6553,22823.5947,5,12.9411,LONG,22850,PE,2025-03-20,1,2025-03-19 09:51:00,69.4000,52.3500,2025-03-19 15:21:00,EOD Exit,17.0500,1.2175,15.8325,10000000,8,3501.0941,55431.0722,0.5543,2025
131,2025-03-20,23055.9035,22992.5465,5,15.7107,SHORT,23000,CE,2025-03-20,0,2025-03-20 10:09:00,47.0500,93.4500,2025-03-20 11:12:00,TSL Hit,-46.4000,1.4050,-47.8050,10000000,8,3478.2609,-166278.2609,-1.6628,2025
132,2025-03-20,23052.5035,22992.5465,5,15.7107,LONG,23050,PE,2025-03-20,0,2025-03-20 11:12:00,31.6000,0.1500,2025-03-20 15:21:00,EOD Exit,31.4500,0.3175,31.1325,10000000,8,3470.7158,108052.0607,1.0805,2025
133,2025-03-21,23201.4350,23147.3650,5,13.7270,LONG,23200,PE,2025-03-27,6,2025-03-21 09:30:00,118.3000,74.3000,2025-03-21 13:36:00,TSL Hit,44.0000,1.9260,42.0740,10000000,8,3448.2759,145082.7586,1.4508,2025
134,2025-03-21,23201.4350,23334.0650,5,13.7270,SHORT,23350,CE,2025-03-27,6,2025-03-21 13:36:00,138.5500,138.5500,2025-03-21 13:36:00,TSL Hit,0.0000,2.7710,-2.7710,10000000,8,3426.1242,-9493.7901,-0.0949,2025
135,2025-03-21,23345.6850,23283.5650,5,13.7270,LONG,23350,PE,2025-03-27,6,2025-03-21 14:09:00,66.4000,60.4500,2025-03-21 15:21:00,EOD Exit,5.9500,1.2685,4.6815,10000000,8,3426.1242,16039.4004,0.1604,2025
136,2025-03-24,23542.7566,23439.3434,5,21.8513,LONG,23550,PE,2025-03-27,3,2025-03-24 10:00:00,119.7500,71.3500,2025-03-24 15:21:00,EOD Exit,48.4000,1.9110,46.4890,10000000,8,3397.0276,157924.4161,1.5792,2025
137,2025-03-25,23754.7928,23684.2572,5,16.3886,SHORT,23700,CE,2025-03-27,2,2025-03-25 09:24:00,139.5000,173.7000,2025-03-25 09:30:00,TSL Hit,-34.2000,3.1320,-37.3320,10000000,8,3375.5274,-126015.1899,-1.2602,2025
138,2025-03-25,23746.8428,23684.2572,5,16.3886,LONG,23750,PE,2025-03-27,2,2025-03-25 09:30:00,135.2500,115.9000,2025-03-25 10:12:00,TSL Hit,19.3500,2.5115,16.8385,10000000,8,3368.4211,56719.1579,0.5672,2025


In [32]:
tb['ROI%'].sum()

30.925624082134096

In [33]:
stats = generate_stats(tb, 'ATR Dynamic')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,0.0000,0,NaN,NaN,NaN,NaN,NaN,ATR Dynamic
2020,0.0000,0,NaN,NaN,NaN,NaN,NaN,ATR Dynamic
2021,0.0000,0,NaN,NaN,NaN,NaN,NaN,ATR Dynamic
2022,0.0000,0,NaN,NaN,NaN,NaN,NaN,ATR Dynamic
2023,0.0000,0,NaN,NaN,NaN,NaN,NaN,ATR Dynamic
2024,0.0000,0,NaN,NaN,NaN,NaN,NaN,ATR Dynamic
2025,30.9256,151,50.3311,1.0861,-0.6883,-9.4909,3.2585,ATR Dynamic
Overall,30.9256,151,50.3311,1.0861,-0.6883,-9.4909,3.2585,ATR Dynamic


In [197]:
stats = generate_stats(tb, 'ATR Dynamic')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,20.0140,444,47.0721,1.2831,-1.1229,-18.0695,1.1076,ATR Dynamic
2020,82.1398,434,47.4654,2.0710,-1.5659,-24.1027,3.4079,ATR Dynamic
2021,89.6891,456,49.3421,1.4456,-1.0332,-21.9868,4.0792,ATR Dynamic
2022,28.3868,427,47.7752,1.4828,-1.2573,-25.2311,1.1251,ATR Dynamic
2023,-10.8861,437,45.0801,0.8915,-0.8180,-20.5832,-0.5289,ATR Dynamic
2024,43.3070,488,48.5656,1.1689,-0.9312,-21.8567,1.9814,ATR Dynamic
2025,33.5247,140,57.8571,0.9741,-0.7691,-6.4436,5.2028,ATR Dynamic
Overall,286.1752,2826,48.0892,1.3643,-1.1003,-25.2311,11.3421,ATR Dynamic


In [198]:
tb.to_csv('ATR_Dynamic_JJMS.csv', index=False)

In [193]:
# SIMULATION

TF_ = ['3m', '5m', '1m', '10m']
ATR_WINDOW_ = [5, 8, 11, 14, 18, 21, 25]
MULTIPLIER_ = [2, 2.5, 3, 3.5, 4, 5, 6, 7]

for i in TF_:
    for j in ATR_WINDOW_:
        for k in MULTIPLIER_:
            if i == '10m':
                z = '5m'
            else:
                z = '0m'
            variation = f'TF: {i}, ATR: {j}, MULT: {k}'
            print(variation)
            bnf_df = resample(pl.DataFrame(bnf_1min), i, z)
            bnf_df_pandas = bnf_df.to_pandas()
            bnf_df_pandas = await add_atr(bnf_df_pandas, j)
            tb = await backtest_intraday_levels(bnf_df_pandas, k, i, z)

            if len(tb) > 0:
                stats = generate_stats(tb, variation)
                for x, y in stats.items():
                    q = pd.DataFrame(y)
                    
                    print(q.to_string())


TF: 3m, ATR: 5, MULT: 2
        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio                Variation
2019      14.2032         1027  39.7274                0.9145             -0.6052     -29.8402       0.4760  TF: 3m, ATR: 5, MULT: 2
2020      59.4733          904  41.4823                1.5520             -1.0107     -35.5439       1.6732  TF: 3m, ATR: 5, MULT: 2
2021      56.2519          980  42.1429                1.0545             -0.6737     -16.3470       3.4411  TF: 3m, ATR: 5, MULT: 2
2022      35.4420          832  42.7885                1.0698             -0.7460     -20.5936       1.7210  TF: 3m, ATR: 5, MULT: 2
2023     -26.8060          917  37.6227                0.7278             -0.5099     -35.0905      -0.7639  TF: 3m, ATR: 5, MULT: 2
2024     -37.5746         1078  40.2597                0.8839             -0.6550     -73.5714      -0.5107  TF: 3m, ATR: 5, MULT: 2
2025       0.7888          304  41.1184      

IndexError: single positional indexer is out-of-bounds